In [3]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
from tensorflow import keras
import h5py
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers

from datagen import DataGenerator

In [4]:
batch_size=32
epochs=50
input_shape = (20,21,3)
classes = 4
train_gen = DataGenerator('dynamic_gestures2.hdf5')

A: [[[ 3.16374004e-01  3.51073503e-01 -1.41977393e-04]
  [ 2.54676700e-01  3.39414895e-01 -9.63216976e-02]
  [ 2.18510449e-01  2.61363447e-01 -1.29139099e-01]
  ...
  [ 3.03846955e-01  8.27390552e-02  5.28673499e-02]
  [ 3.01035404e-01  4.12825346e-02  4.96314845e-02]
  [ 2.96733379e-01  1.37305260e-03  4.54961366e-02]]

 [[ 1.46659613e-02  3.93791080e-01 -7.58155384e-05]
  [-7.40879774e-02  3.67804050e-01 -3.65879156e-02]
  [-1.33595705e-01  2.83198595e-01 -5.71323664e-02]
  ...
  [ 4.25117612e-02  9.64299440e-02 -8.56183649e-02]
  [ 4.19123769e-02  4.74910140e-02 -1.08550812e-01]
  [ 3.89934182e-02 -7.29441643e-04 -1.27618510e-01]]

 [[ 1.54061317e-02  3.96268547e-01 -6.57246073e-05]
  [-7.36961961e-02  3.70012164e-01 -3.31853063e-02]
  [-1.32648468e-01  2.84559488e-01 -5.18045009e-02]
  ...
  [ 4.66657877e-02  1.01710618e-01 -8.67992521e-02]
  [ 4.61905003e-02  5.14893532e-02 -1.11019749e-01]
  [ 4.33127284e-02  2.22086906e-03 -1.30594525e-01]]

 ...

 [[ 2.95482278e-01  3.64222527e

In [7]:
inputs = layers.Input(shape=input_shape)
x = inputs
x = layers.TimeDistributed(layers.Flatten())(x)
x = layers.TimeDistributed(layers.Dropout(0.5))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.TimeDistributed(layers.Dense(64, activation='relu'))(x)
x = layers.TimeDistributed(layers.Dropout(0.2))(x)
x = layers.GRU(32)(x)
output = layers.Dense(classes, activation='softmax',name='Output')(x)
model = keras.models.Model(inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20, 21, 3)]       0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 20, 63)            0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 20, 64)            4096      
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 20, 64)            4160      
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 64)            0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 20, 64)            4160

In [6]:
model.fit_generator(generator=train_gen,epochs=epochs)

C:\Python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
Epoch 1/50
69/69 [==============================] - 4s 6ms/step - loss: 1.0851 - accuracy: 0.5964
Epoch 2/50
69/69 [==============================] - 0s 6ms/step - loss: 0.1248 - accuracy: 0.9725
Epoch 3/50
69/69 [==============================] - 0s 6ms/step - loss: 0.0407 - accuracy: 0.9911
Epoch 4/50
69/69 [==============================] - 0s 6ms/step - loss: 0.0222 - accuracy: 0.9964
Epoch 5/50
69/69 [==============================] - 0s 6ms/step - loss: 0.0144 - accuracy: 0.9979
Epoch 6/50
69/69 [==============================] - 0s 7ms/step - loss: 0.0090 - accuracy: 0.9984
Epoch 7/50
69/69 [==============================] - 0s 7ms/step - loss: 0.0142 - accuracy: 0.9960
Epoch 8/50
69/69 [=================

In [23]:
model.save("test_model_2", save_format = "tf")

INFO:tensorflow:Assets written to: test_model_2\assets


In [4]:
!pip --version

pip 19.1.1 from C:\ProgramData\Anaconda3\lib\site-packages\pip (python 3.7)



In [3]:
x,y=[],[]
with h5py.File('dynamic_gestures2.hdf5', 'a') as f:
    print(f.filename)
    i=0
    for _,gesture in enumerate(list(f['/'])):
        if not "swipe" in gesture:
            continue
        print(i,gesture)
        data = list(f['/'+gesture])
        x.extend(data)
        y.extend([i]*len(data))
        i+=1
        if gesture == "swipe_right":
            print(i,"swipe_left")
        elif gesture == "swipe_up":
            print(i,"swipe_down")
        data = list(np.flip(np.array(list(f['/'+gesture])), axis=1))
        x.extend(data)
        y.extend([i]*len(data))
        i+=1

x=np.array(x)
y=np.array(y) 
y=tf.keras.utils.to_categorical(y,classes)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,shuffle=True)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

dynamic_gestures2.hdf5
0 swipe_right
1 swipe_left
2 swipe_up
3 swipe_down
(669, 20, 21, 3) (75, 20, 21, 3) (669, 9) (75, 9)
